In [1]:

# Import the dependencies.
import pandas as pd
import gmaps
import requests



# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Fortuna,US,2022-08-14 15:58:38,40.5982,-124.1573,61.66,92,100,5.75
1,1,Manaure,CO,2022-08-14 15:58:38,11.7751,-72.4445,86.85,61,86,8.81
2,2,Umm Lajj,SA,2022-08-14 15:58:39,25.0213,37.2685,87.57,62,99,9.53
3,3,Port Lincoln,AU,2022-08-14 15:58:39,-34.7333,135.8667,50.70,82,93,8.55
4,4,Chapais,CA,2022-08-14 15:58:40,49.7834,-74.8492,69.58,51,99,3.31


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
#get the maximum temperature
max_temp =city_data_df['Max Temp']

temps=[]
for temp in max_temp:
    temps.append(max(temp,0))


In [5]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]

# 2. Assign the weights variable to some values.
temperatures = max_temp = city_data_df["Max Temp"]

# 3. Assign the figure variable to the gmaps.figure() attribute. zoom the map in and center
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=temps,dissipating=False,max_intensity=300,point_radius=2)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)


# 6. Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity

locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=2)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness

locations = city_data_df[["Lat", "Lng"]]

clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=2)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of windspeed

locations = city_data_df[["Lat", "Lng"]]

wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=2)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#ask the user to as min and max temperature values.

min_temp=float(input('What is the minimum temperature you would like for the trip?'))

max_temp=float(input('What is the maximum temperature you would like for the trip?'))


What is the minimum temperature you would like for the trip?70
What is the maximum temperature you would like for the trip?90


In [13]:
#Filter the dataset to find cities that fit the criteria

preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & \
                                    (city_data_df['Max Temp']>= min_temp)].dropna()


preferred_cities_df.count()

City ID       308
City          308
Country       308
Date          308
Lat           308
Lng           308
Max Temp      308
Humidity      308
Cloudiness    308
Wind Speed    308
dtype: int64

In [14]:
#create a new dataframe for hotels and markers off of the preferred cities df

hotel_df=preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()

hotel_df['Hotel Name']=""

hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Manaure,CO,86.85,11.7751,-72.4445,
2,Umm Lajj,SA,87.57,25.0213,37.2685,
7,San Patricio,US,79.84,28.0170,-97.5169,
9,Torbay,CA,71.17,47.6666,-52.7314,
11,Jawar,IN,74.68,23.0000,76.5000,
12,Elizabeth City,US,80.02,36.2946,-76.2511,
13,Jamestown,US,75.94,42.0970,-79.2353,
15,Kavieng,PG,80.60,-2.5744,150.7967,
17,Gagnoa,CI,76.57,6.1293,-5.9437,
18,Atuona,PF,76.84,-9.8000,-139.0333,


In [16]:
# Set parameters to search for a hotel.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location":""
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found...skipping')
        

    
hotel_df.head()

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Manaure,CO,86.85,11.7751,-72.4445,Hotel Palaaima
2,Umm Lajj,SA,87.57,25.0213,37.2685,Moon light Furnished Units
7,San Patricio,US,79.84,28.0170,-97.5169,"Motel 6 Sinton, TX"
9,Torbay,CA,71.17,47.6666,-52.7314,Gallows Cove
11,Jawar,IN,74.68,23.0000,76.5000,Sree balaji


In [18]:
hotel_df.head(100)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Manaure,CO,86.85,11.7751,-72.4445,Hotel Palaaima
2,Umm Lajj,SA,87.57,25.0213,37.2685,Moon light Furnished Units
7,San Patricio,US,79.84,28.0170,-97.5169,"Motel 6 Sinton, TX"
9,Torbay,CA,71.17,47.6666,-52.7314,Gallows Cove
11,Jawar,IN,74.68,23.0000,76.5000,Sree balaji
...,...,...,...,...,...,...
182,Sfantu Gheorghe,RO,70.93,45.8667,25.7833,Hotel Park
185,Umm Kaddadah,SD,85.98,13.6017,26.6876,
186,Port-Gentil,GA,76.96,-0.7193,8.7815,Fengshui Residence
190,Uvarovka,RU,74.03,55.5278,35.6061,


In [19]:
# Add a heatmap of temperature for the vacation spots.

locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Add a heatmap of temperature for the vacation spots and marker for each city.

locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))